# OpenAI Function Calling


**Notes**:
- LLM's don't always produce the same results. The results you see in this notebook may differ from the results you see in the video.
- Notebooks results are temporary. Download the notebooks to your local machine if you wish to save your results.

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

**Making function that will return the wheater value**

In [2]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

**So now we will use new feature of openai which is "functions" to specify that it has to hit above function.**

In [3]:
# define a function
functions = [
    {
        #name of the function that will be called
        "name": "get_current_weather",
        
        #this description is important because it will be passed to LLM
        "description": "Get the current weather in a given location",
        
        
        #************************************************************
        #these parameters will be obtained from the prompot and properties and desciption of properties will help LLM do that
        #In parameters part we will specify all those things that will help LLM to get parameters from the prompt
        #also thorugh this parameter information LLM will figure out whether to use the specifies function or not
        "parameters": {
            #parameter will be of type object
            "type": "object",
            #-----------------------------------------------
            #our parameter will have following properties
            "properties": {
                "location": {
                    "type": "string",
                    
                    #this desciption is also very important because it will be passed directly to language model used
                    #as we will pass location for the wheather so we will give description according to it
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                #secondly temperature unit
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            #---------------------------------------------
            
            #the required parameter to hit the function is location (we use required parameter to make LLM understand that if it has location, hit the function)
            "required": ["location"],
        },
        #**************************************************************
    }
]

In [6]:
#message that will be passed to LLM
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [7]:
import openai

In [8]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    #as we are specifying funcitons here
    functions=functions
)

In [9]:
print(response)

In [10]:
response_message = response["choices"][0]["message"]

In [11]:
response_message

In [12]:
response_message["content"]

In [13]:
response_message["function_call"]

In [ ]:
json.loads(response_message["function_call"]["arguments"])

In [ ]:
args = json.loads(response_message["function_call"]["arguments"])

In [ ]:
get_current_weather(args)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [ ]:
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

In [ ]:
messages.append(response["choices"][0]["message"])

In [ ]:
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
observation = get_current_weather(args)

In [ ]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)